In [46]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import keras

In [47]:
df = pd.read_csv('data/preped-smiles.smi.csv')
df1 = pd.read_csv('data/unique_smile_chars.csv')

df.head()

,0
0,C1=Cc2cc1nnc1snc(o2)-o-1
1,N1C2C3C4C5NC6C7C6C5(C13)C2N47
2,c1c2c[nH]c(nn3cnc(c#1)c3)-s-2
3,N=c1-c2cnn-1cnccc(=O)c2
4,C=Nn1-c2cccconc-1[nH]c2


In [113]:
smiles = df.values.tolist()
chars = df1.values.tolist()

smile_len = [len(j[0]) for j in smiles]
maxlen = max(smile_len)
step = 3

print(maxlen)
print(smiles[0][0])
print(len(chars))

36
C1=Cc2cc1nnc1snc(o2)-o-1
29


In [118]:
smiles_dict = {}
a=0
for k in smiles:
    smiles_dict[a] = k[0]
    a+=1

chars_dict = {}
a=0
for k in chars:
    chars_dict[a] = k[0]
    a+=1
    
print(len(smiles_dict))
print(len(chars_dict))

1000000
29


In [126]:
next_chars = []
for idx, val in smiles_dict.items():
    if idx < len(smiles_dict)-1:
        next_chars.append(smiles_dict[idx+1][0][0])
    else:
        next_chars.append(smiles_dict[0][0][0])

print(len(next_chars))
print(next_chars[999999])

1000000
C


In [133]:
# ONE HOT ENCODING
print('Vectorization...')

char_indices = dict((char, idx) for idx, char in chars_dict.items())

x = np.zeros((len(smiles), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(smiles), len(chars)), dtype=np.bool)

for i, smile in enumerate(smiles):
    for t, char in enumerate(smile[0]):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [136]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [138]:
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [141]:
import random
import sys
for epoch in range(1, 60):
    print('epoch', epoch)
    
    model.fit(x, y, batch_size=128, epochs=1)
    #start_index = random.randint(0, len(text) - maxlen - 1)
    #generated_text = text[start_index: start_index + maxlen]
    
    start_index = random.randint(0, len(smiles)-1)
    generated_text = smiles[start_index][0]
    
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index][0]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
1000000/1000000 [==============================] - 750s 750us/step - loss: 0.5472
--- Generating with seed: "CCOC(=O)CONC1=NCCO1"
------ temperature: 0.2
CCOC(=O)CONC1=NCCO1CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC------ temperature: 0.5
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCOCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

C:\Users\EAZETNG\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC------ temperature: 0.5
CCCCCCCCCCCCCCCCCCCCCCNCCCCCCCCCCCCCCCNCCNCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCOCCCCCCCCCCCCCCCCCCCCCCCCCCCCNCCCCCCNCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCNCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCNCCCCCCCCOCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC------ temperature: 1.0
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCOCCCCCCCCCOCCCCCCCCOCCCCCCCNCCCCOCCCCCCCCOCCCCCCCCCCCNCCCCCCOCCCCCCOCCCNCCCCCCCCCNCCCCCCCCOCCCCCCC

CCCCCCCCCCCCCCCCCCCCCCCCCCCOCCCCCCCCCCCCCCCCCCCCOCCCCCOOCCCCCCCCCCCCCNCCOCCCCCCOCCCCCCCCCCCCCCCCCCOCCCCCCCCCCCCCCCCCCCCCCCCCCCCCOOCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCNCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCOCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCOCCCCCCCCCCCCCCNCCCCCCCCCCCCCCCCCCONCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCOCCCCCCCCCCCCCCCCCCCCCOCCCCCCCCCCCCCCCCCCCCCCCCC------ temperature: 1.0
CCCCCCCCCCCCCCCCCCCCCNCCCCCNCCOCCCCCCCCNCCCCCCOONCCCCCCCCCCONONCCCCCCCCCCCCNCCCCCCCCCCOCCNCCCCNOCCCCCCNOCCCCCOCCONCCCCCCCONCCCNCCCCCOCCCCCCCCOCOCNCOCCCCCCCCCCNNNCCCCNCCCCCCCCNCCCCCCNCCNCCCCCCCCCCCCCOCCCCCCCONOCCCCCCCCNCCNCCCCCNCCCCCCCCCCCCCOOCCOOCCCCCOCCCOCCOCCCCCCCCOCCOCCCCOCCCCOCONCCCCCCCCCCCNCCCCCCCCCOCCCCCCNCCCCCONNCCCCCCOCCCNCCONCCCCCCCCCCCCCCCNCCCOCCOCCCCCOCCCCCCCCCCCCCCCCCOCCCCCCCCCOCCCNCCCCCCCCCCCCOCOCCCCCCC------ temperature: 1.2
CCCCCCCCCOCOCCCCCCCCCOONOCOCCCCCCCNCCCCNOCOCCOCCCCCCCCCOCOCCNOCCCCCCCCCCOCCCNCOOCOCCCCCONOCCCCOCNCCCNCCCCCCNONOCCO

KeyboardInterrupt: 